# Preprocesamiento

Para la utilización de diversos modelos debemos hacer un preprocesamiento de los textos que queremos analizar. Vamos a utilizar **Gensim** para hacer esta fase. 

Los pasos que vamos a seguir son los siguientes:

- Cargar un conjunto de documentos.
- Normalizar los documentos, quitando las palabras comunes y los caracteres extraños.
- Construir un diccionario.

### Construcción de un diccionario

Comenzamos construyendo una lista con palabras comunes que no vamos a querer tener en cuenta en el análisis. Utlizamos la librería *codecs* para no tener problemas respecto de la codificación de los documentos, además de agregar como *header* el *encoding* explícitamente para no tener problemas al momento de escribir comentarios en el script o imprimir en pantalla palabras con tildes o caracteres raros. Las palabras comunes están especificadas en el archivo *stopwords_spanish* (separadas por el caracter *\r\n*).

In [1]:
#! -*- coding: utf8 -*-

# Carga de palabras comunes mediante la librería codecs
import codecs 
stopwords_file = 'stopwords_spanish.txt'
stopwords = codecs.open(stopwords_file,'r','utf8').read().split('\r\n')

Inspeccionamos la lista *stopwords* para ver algunos de los términos cargados:

In [2]:
print(stopwords[:10])

[u'a', u'actualmente', u'ac\xe1', u'ademas', u'adem\xe1s', u'adrede', u'afirm\xf3', u'agreg\xf3', u'ahi', u'ahora']


La *u* delante de cada *string* especifica el [*encoding Unicode*](https://es.wikipedia.org/wiki/Codificaci%C3%B3n_de_caracteres) que maneja sin problemas todo tipo de caracteres (muchas veces cuando se escribe en python una frase con tildes hay que agregar la *u* delante, por ejemplo, u'además'). Al imprimir la lista entera, los términos se ven efectivamente codificados como *Unicode*, pero al imprimir en pantalla término a término vemos que todo funciona correctamente:

In [3]:
print(stopwords[2])

acá


Cargamos ahora los textos que vamos a utlizar tanto para la construcción del diccionario como para el análisis. Muchas veces va a ser de preferencia construir un diccionario (o mejor un modelo **Tf-idf** que veremos más adelante) con un corpus más grande que el conjunto de textos que vamos a analizar, así que no necesariamente debe coincidir el conjunto de textos utilizados para el preprocesamiento que para el análisis.

Por el momento definimos la lista de documentos mediante una una [lista por compresión](http://elclubdelautodidacta.es/wp/2013/04/python-listas-por-comprension-1/):

In [4]:
# Número total de documentos
ndocs = 400

# Carga de documentos
documents = [codecs.open('Data/file{}.txt'.format(i),'r','utf8').read() for i in range(ndocs)]

Podemos ver el primer documento llamando al primer elemento de la lista definida anteriormente. Recordar que en python el primer elemento se denomina con el índice 0. Para no observar todo el documento entero vemos hasta el caracter 1000 incluido (que tiene el índice 999 ya que se cuenta desde 0):

In [5]:
print(documents[0][:1000])

Varios famosos respaldaron a Muriel Santa Ana luego de haber contado que se realizó un aborto
Facundo Arana intentó parecer un caballero, pero terminó desencadenando un alud de críticas. El actor aseguró, en medio de una entrevista, que  a su ex, Isabel Macedo, la veía doblemente realizada como mujer, primero por haber encontrado al hombre ideal, y después por estar a punto de dar a luz a su primer hijo . Las palabras del actor no pasaron desapercibidas para muchas de sus colegas, que salieron a retrucar sus dichos en Twitter. Una de las más enérgicas fue          Muriel Santa Ana  , quien contó que no está en sus planes casarse ni tener hijos, además de confesar que a los 24 años se realizó un aborto. Esta revelación generó una nueva polémica en las redes y en los medios, evidenciando una vez más que el derecho al aborto sigue siendo un tema que divide las aguas y que merece ser discutido y tratado en profundidad. Una de las primeras en brindar su apoyo a Santa Ana fue su amiga,      

El siguiente paso es separar cada documento por palabras (es decir, descripto como 1-gramas. Bien podría extenderse a n-gramas). **Gensim** tiene la función *tokenize* que además tiene incorporadas otras funcionalidades como minimizar todas las letras mediante *lowercase = TRUE*, y elimina todos los términos que no están compuestos exclusivamente por caracteres alfabéticos, **incluyendo los números**. En caso de querer conservar los números se recomienda utilizar la función *word_tokenize* de la librería de análisis de lenguaje natural **Nltk**:

In [6]:
from gensim.utils import tokenize

# Separación en palabras y minimización de las palabras
documents = [list(tokenize(doc, lowercase = True)) for doc in documents]

Por último, filtramos las palabras comunes presentes en la lista *stopwords* mediante la función *filter*. El criterio de la función *filter* como muchas de otras funciones de python mediante definiendo [funciones *lambda*](https://recursospython.com/guias-y-manuales/funciones-lambda/):

In [7]:
# Normalización y limpieza de documentos: se descartan las palabras comunes.
documents = [filter(lambda x: x not in stopwords, doc) for doc in documents]

Listo! Tenemos los documentos dentro de un solo objeto, cada uno de ellos definidos a su vez como una lista cuyos elementos son todas la palabras minimizadas compuestas exclusivamente por caracteres alfabéticos y que no estaban incluídas dentro de la lista *stopwords*. 
Podemos ver cómo quedó descrito el primer documento llamando nuevamente al primer elemento de la lista *documents*. En este caso inspeccionamos los primeros 10 elementos:

In [8]:
print(documents[0][:10])

[u'famosos', u'respaldaron', u'muriel', u'santa', u'ana', u'contado', u'aborto', u'facundo', u'arana', u'intent\xf3']


El paso final es construir con todos estos términos un diccionario. Con **Gensim** esto se puede hacer fácilmente mediante la clase *Dictionary*, además de poder guardarlo para utilizarlo posteriormente.

In [9]:
from gensim.corpora import Dictionary

# Construccion del diccionario con gensim
dictionary = Dictionary(documents)

# Guardado del diccionario
dictionary.save('tutorial.dict')

### Inspección del diccionario

El diccionario es un objeto el cual tenemos que tener siempre presente para consultar la relación entre índices y términos, y muchas veces es requerido como argumento para la construcción de muchos modelos tales como LSA, LDA, etc.
Por ejemplo, podemos inspeccionar cuál es el primer y el quinto término cargado en el diccionario:

In [10]:
print u'1° Término: ', dictionary[0]
print u'5° Término: ', dictionary[4]

1° Término:  aborto
5° Término:  acompañó


En forma inversa podemos ver cuál es el índice asociado a un determinado término mediante el atributo *token2id* (que es a su vez un [*diccionario* de python](http://entrenamiento-python-basico.readthedocs.io/es/latest/leccion3/)), en caso que esté incorporado en el diccionario. Por ejemplo, buscamos los índices asociados a los términos *aborto* y *macri*:

In [11]:
print 'aborto: ', dictionary.token2id['aborto']
print 'macri: ', dictionary.token2id['macri']

aborto:  0
macri:  1031


Otra gran utilidad del diccionario es su capacidad de convertir los textos en una bolsa de palabras o *bow* (*bag of words*). Una bolsa de palabras consiste en describir los documentos específicando los términos que aparecen y la frecuencia con la que lo hacen. Tomemos por ejemplo la siguiente frase:

In [12]:
example = u'Despenalización del aborto: argumentos a favor y en contra de la despenalización.'

Necesitamos hacer siempre el preprocesamiento de minimizar la frase y dividirlo en palabras:

In [13]:
example = list(tokenize(example.lower()))
print(example)

[u'despenalizaci\xf3n', u'del', u'aborto', u'argumentos', u'a', u'favor', u'y', u'en', u'contra', u'de', u'la', u'despenalizaci\xf3n']


El siguiente paso es invocar el método *doc2bow* del diccionario:

In [14]:
# Ejemplo descrito como una bag of words

bow = dictionary.doc2bow(example)
print(bow)

[(0, 1), (75, 1), (575, 1), (1267, 2)]


Esta descripción es una lista de tuplas del estilo (índice del término,frecuencia de aparación). La lista de tuplas es una forma más eficiente de describir el documento, como alternativa a escribir en forma completa el vector documento en el espacio de términos. Dado que un documento solo contiene unos pocos términos en relación con la cantidad de términos presentes en el diccionario, el vector (y la matriz si pensamos en todo el corpus) es muy *sparse*, por lo tanto solo se indican los términos que tienen frecuencias no nulas.

Vemos cómo queda entonces la descripción del ejemplo:

In [15]:
for b in bow:
    print(u'Palabra: {}. Frecuencia: {}'.format(dictionary[b[0]], b[1]))

Palabra: aborto. Frecuencia: 1
Palabra: favor. Frecuencia: 1
Palabra: argumentos. Frecuencia: 1
Palabra: despenalización. Frecuencia: 2


Notar que las palabras comunes fueron descartadas al no incluirlas originalmente en el diccionario.

### Salvado del corpus como *bag of words*

Finalmente utilizemos el diccionario para dar una descripción del tipo *bag of words* de todo el corpus. Guardamos este objeto con la librería *cPickle* que nos permite guardar y cargar cualquier objeto creado en python:

In [16]:
# Corpus como bag of words
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Salvamos el corpus
import cPickle as pk
pk.dump(corpus, file('Tutorial_corpus.pk','w'))

Podemos cargar el corpus más adelante mediante el siguiente comando:

In [17]:
corpus = pk.load(file('Tutorial_corpus.pk','r'))

### Term Frequency - Inverse document frequency (Tf-idf)

Cuando uno describe un documento como una bolsa de palabras podemos tener el problema que las palabras que aparecen con mucha frecuencia tienen mucho peso en el documento y aún así ser no representativas. Si bien ya realizamos un filtrado de palabras comunes como preposiciones o artículos este problema está latente. 
Para solucionarlo podemos aplicar el algoritmo **TF-idf** al corpus descrito como *bag of words* que le da más peso a aquellas palabras que son muy específicas e importantes para la definición de un tópico por ejemplo, pero aún así no son palabras citadas frecuentemente. Por otro lado permite quitarle importancia a aquellas palabras que son comunes en diversos documentos, y por lo tanto lograr mayor especificidad.

**Gensim** calcula el peso del término $i$ en el documento $j$ mediante la siguiente fórmula, donde $f_{ij}$ es la frecuencia con la que aparece el $i$ en $j$ y $df_{i}$ (*document frequency*) es la cantidad de documentos donde aparece el término $i$:

$$ w_{ij} = f_{ij} \cdot log_2 \frac{D}{df_{i}} $$

Notar que si un término aparece en todos los documentos del corpus su peso solo estará dado por la frecuencia con la que aparece en cada documento. En cambio si un término es muy específico para un subconjunto de documentos tendrá en ellos un peso mayor.

Por otro lado, una vez recalculados los pesos de cada término, se suele normalizar los vectores a norma euclidea 1. Por defecto **Gensim** lo hace, ya que puede ayudar a mejorar la comparación entre textos de distinto largo. Esta opción también puede tenerse en cuenta cuando se construye el corpus como *bag of words*. En ambos casos depende de en qué algoritmo vamos a aplicar nuestra descripción del corpus y ahí tomar la desición de utilizar la normalización o no.

Apliquemos **Tf-idf** en **Gensim** sobre nuestro corpus:

In [18]:
from gensim.models import TfidfModel

# Construimos el modelo tfidf con todo el corpus
tfidf = TfidfModel(corpus, normalize=True)

# Transformamos el corpus pesando cada componente con la fórmula tfidf
corpus_tfidf =  tfidf[corpus]

Notar que en las líneas anteriores podríamos haber construido el modelo **tfidf** con todo el corpus y luego transformar un porción más pequeña, si solo nos interesa analizar una parte de él. Cuanto más textos tenga el corpus con el que entrenamos el modelo, se destacarán más los términos específicos.

Podemos ver como transforma el modelo **tfidf** en la frase que dimos de ejemplo:

In [19]:
example = u'Despenalización del aborto: argumentos a favor y en contra de la despenalización.'
example = list(tokenize(example.lower()))
bow = dictionary.doc2bow(example)
print(tfidf[bow])

[(0, 0.24405263283153336), (75, 0.32262003197123384), (575, 0.590180434039706), (1267, 0.6985998014998152)]


La valorización de cada término depende mucho del corpus con el que entrenamos el modelo **tfidf**.

Guardamos finalmente el corpus transformado con **tfidf** para utilizarlo posteriormente. Muchos objetos de **Gensim** tienen el método *save* incorporado, pero podemos utilizar nuevamente el módulo *cPickle*:

In [20]:
pk.dump(corpus_tfidf, file('Tutorial_corpus_tfidf.pk','w'))

Para cargarlo utilizamos el método *load*:

In [21]:
corpus_tfidf = pk.load(file('Tutorial_corpus_tfidf.pk','r'))